# Auxiliar 4, parte 1
Esta auxiliar va a tener 3 partes, primero vamos a volver a visitar el ejemplo de clasificación de documentos que vimos en la auxiliar pasada, pero esta vez usaremos algunas de las utilidades que nos ofrece pytorch para facilitar el trabajo. Además, probaremos cargando embeddings preentrenados para ver como afecta esto al desempeño de nuestro modelo. Las otras dos partes son ejemplos de CNN y de RNN, pero lo voy a separar en varios notebooks para que quede mas organizado.

Esta clase sera principalmente práctica, así que la mayor parte de las explicaciones las voy a incluir en el código.

In [1]:
%%capture --no-stderr
!pip install --upgrade torchtext
!pip install --upgrade tqdm # resuelve un problema de mas adelante (?)

In [2]:
# Si quieren tener resultados reproducibles (obtener los mismos resultados
# al correr el notebook varias veces) es necesario setear la "semilla" para
# las librerias que general los numeros aleatorios usados en el script.
# Esta aleatoriedad puede aparecer al crear splits aleatorios de un dataset,
# al inicializar los pesos de una red, al agregar una capa de dropout, etc.

# Pueden leer un poco mas acerca de esto aca
# https://pytorch.org/docs/stable/notes/randomness.html
import random
import torch
torch.manual_seed(8888)
random.seed(8888)
torch.backends.cudnn.deterministic = True

In [3]:
import csv
import gzip
import os
import shutil

import requests

# Descargar el dataset, lo mismo que antes, solo que ahora lo voy a
# guardar en un archivo porque la API de torch text usa
# archivos y no file-like objects :(

# Leer los datos de esta forma permite realizar filtrado y modificaciones
# a medida que se van stremeando los datos de la red. Esto no es para nada
# necesario, y podrian usar algo como wget para descargar los datos y luego
# hacer preprocesamiento mas adelante en su codigo.
DATASET_FILE = "argument_mining.tsv"
GLOVE_FILE = "glove300d.vec"
if not os.path.exists(DATASET_FILE):
    print(f"Descargando {DATASET_FILE}")
    # Descargar los datos directamente de github
    url = "https://github.com/uchile-nlp/ArgumentMining2017/raw/master/data/complete_data.csv.gz"
    # Usar stream=True para que no se descarguen todos los datos a la vez
    response = requests.get(url, stream=True)
    try:
        with open(DATASET_FILE, "w") as out_file:
            writer = csv.writer(out_file, delimiter="\t")
            for row in csv.DictReader(
                # Descomprimir a medida que se leen los datos de la red
                gzip.open(response.raw, mode="rt"),
                strict=True,
                escapechar="\\",
            ):
                # Nos quedamos solamente con el primer topico para efectos
                # del ejemplo
                if row["topic"] == "1" and row["argument"]:
                    # Solamente las columnas que nos interesan del dataset
                    writer.writerow(
                        [row["constitutional_concept"], row["argument"]]
                    )
    except Exception as e:
        os.remove(DATASET_FILE)
        raise e

# Descargar vectores glove de aca
# https://github.com/dccuchile/spanish-word-embeddings
if not os.path.exists(GLOVE_FILE):
    print(f"Descargando {GLOVE_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(GLOVE_FILE, "wb") as f_out:
                # Funcion util para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(GLOVE_FILE)
        raise e


In [4]:
%%capture --no-stderr
!python -m spacy download es

In [5]:
from string import whitespace, punctuation
from spacy.lang.es.stop_words import STOP_WORDS
from torchtext import data

# Para cargar un dataset en torchtext primero deben definir
# los campos del dataset (Fields), donde tambien se guarda informacion
# asociada a este campo. Por ejemplo, si quieren que se tokenize (los 
# campos de tipo `sequential` son tokenizados), que tokenizer usar,
# as stop_words que seran ignoradas luego al crear el vocabulario, etc.
# Son estos mismos Fields los que se usan luego para crear sus vocabularios
# asociados

# La documentacion sobre un Field la pueden encontrar aca
# https://pytorch.org/text/data.html#torchtext.data.Field

LABEL = data.Field(
    sequential=False, pad_token=None, unk_token=None, is_target=True
)
TEXT = data.Field(
    tokenize="spacy",
    tokenizer_language="es",
    stop_words=STOP_WORDS.union(whitespace, punctuation),
    lower=True,
    batch_first=True,
)

# Ahora con los Fields definidos es super simple cargar el dataset, le dan
# la ubicacion del dataset, el tipo de separacion entre columnas y los fields
# que definieron y torchtext sabe que hacer para separar el dataset en esos
# fields.
dataset = data.TabularDataset(
    path="argument_mining.tsv",
    format="tsv",
    fields=[("label", LABEL), ("text", TEXT)],
)


In [18]:
from torchtext import vocab

# Para cargar los vectores de embeddings (que son esencialmente un vocabulario
# donde cada palabra tiene asociado un vector) pueden usar la clase vocab.Vectors
es_embeddings = vocab.Vectors(GLOVE_FILE)

100%|██████████| 855380/855380 [01:57<00:00, 7309.86it/s]


In [19]:
from operator import attrgetter

# Para construir el vocabulario es muy simple, llaman build_vocab para cada
# Field y le pasan al metodo la el dataset de donde se debe obtener el
# vocabulario para este field. Tambien le pueden pasar varios datasets y el
# Field va a obtener el vocabulario de todos ellos
LABEL.build_vocab(dataset)
TEXT.build_vocab(dataset)

# Para asociarle a los tokens vectores de embedding a partir de una lista
# de embeddings pueden usar el metodo .set_vector sobre el vocab del Field.
# Este metodo recibe 3 argumentos, un mapeo del token al indice dentro de la
# lista de embeddings que corresponde a dicho token, la lista de vectores
# de embedding y la dimension de los embeddings
TEXT.vocab.set_vectors(*attrgetter("stoi", "vectors", "dim")(es_embeddings))

In [20]:
from pprint import pprint
# Mostremos algunos ejemplos
pprint(LABEL.vocab.itos[:10])
print()
pprint(TEXT.vocab.freqs.most_common(10))

['Justicia',
 'Respeto / Conservación de la naturaleza o medio ambiente',
 'Igualdad',
 'Democracia',
 'Descentralización',
 'Bien Común / Comunidad',
 'Respeto',
 'Dignidad',
 'Autonomía / Libertad',
 'Equidad de género']

[('y', 43237),
 ('a', 22099),
 ('respeto', 5656),
 ('derechos', 5592),
 ('igualdad', 5417),
 ('personas', 5282),
 ('sociedad', 5047),
 ('país', 4266),
 ('constitución', 3714),
 ('desarrollo', 3665)]


In [22]:
import torch.nn as nn

# Aca sedefine la arquitectura, para esto usamos una capa de embedding a traves
# de nn.EmbeddingBag, que al pasarle una frase, pasa los tokens por la lista
# de embeddings y los agrega segun `mode` (en este caso los promedia).
# Usamos una capa lineal para hacer clasificacion.
class ArgumentClassifier(nn.Module):
    def __init__(self, embedding_weights, num_class, freeze_embeddings=True):
        super().__init__()
        self.embedding = nn.EmbeddingBag.from_pretrained(
            embedding_weights.clone(), freeze=freeze_embeddings, mode="mean"
        )
        self.fc = nn.Linear(embedding_weights.shape[1], num_class)

    def forward(self, text):
        # text -> (B, N)
        embedded = self.embedding(text)
        return self.fc(embedded)


In [24]:
# Funciones de entrenamiento de toda la vida. Noten que no es necesario
# para los tensores `text` y `labels` a la GPU porque ya estan ahi. Esto
# es porque los iteradores de torchtext permiten definir un device para
# que los tensores sean trasladados automaticamente.
def train_func(train_iter):
    train_loss, train_acc, n_total = 0, 0, 0
    for text, labels in train_iter:
        optimizer.zero_grad()
        output = model(text)
        loss = criterion(output, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(dim=1) == labels).sum().item()
        n_total += len(labels)

    return train_loss / n_total, train_acc / n_total


def test_func(test_iter):
    test_loss, test_acc, n_total = 0, 0, 0
    for text, labels in test_iter:
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, labels)
            test_loss += loss.item()
            test_acc += (output.argmax(dim=1) == labels).sum().item()
            n_total += len(labels)

    return test_loss / n_total, test_acc / n_total


In [25]:
import time

# Hyperparametros de la red
N_EPOCHS = 6
LEARN_RATE = 4.0
BATCH_SIZE = 64

# Esta parte permite que el codigo corra aunque no haya cuda disponible
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instanciamos el modelo y lo pasamos al device correspondiente
model = ArgumentClassifier(
    embedding_weights=TEXT.vocab.vectors, num_class=len(LABEL.vocab),
).to(device)

# Instanciamos el criterio que escogimos (en este caso CrossEntropyLoss)
# y el optimizer (la forma en la que se actualizan los parametros)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)


# Aca usamos el BucketIterator y creams iteradores a partir de splits. Usamos
# el metodo split() del dataset para separarlo en 2 splits. Noten que podemos
# definir un `device` en esta etapa.
train_iter, val_iter = data.BucketIterator.splits(
    dataset.split(0.7, stratified=True, strata_field="label"),
    batch_sizes=(BATCH_SIZE, 256),
    shuffle=True,
    device=device,
    sort=False, # si no ordena los que no son train (esto creo que es una falla de disenno en torchtext)
)


# Esto es lo de toda la vida para entrenar y reportar resultados por epoca
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_iter)
    valid_loss, valid_acc = test_func(val_iter)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    print(
        f"\tLoss: {valid_loss:.4f}(valid)\t|"
        f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    )


Epoch: 1  | time in 0 minutes, 1 seconds
	Loss: 0.0407(train)	|	Acc: 37.2%(train)
	Loss: 0.0097(valid)	|	Acc: 44.8%(valid)
Epoch: 2  | time in 0 minutes, 1 seconds
	Loss: 0.0331(train)	|	Acc: 48.7%(train)
	Loss: 0.0089(valid)	|	Acc: 46.9%(valid)
Epoch: 3  | time in 0 minutes, 1 seconds
	Loss: 0.0308(train)	|	Acc: 51.7%(train)
	Loss: 0.0084(valid)	|	Acc: 49.7%(valid)
Epoch: 4  | time in 0 minutes, 1 seconds
	Loss: 0.0296(train)	|	Acc: 52.9%(train)
	Loss: 0.0082(valid)	|	Acc: 50.6%(valid)
Epoch: 5  | time in 0 minutes, 1 seconds
	Loss: 0.0287(train)	|	Acc: 54.1%(train)
	Loss: 0.0079(valid)	|	Acc: 51.7%(valid)
Epoch: 6  | time in 0 minutes, 1 seconds
	Loss: 0.0282(train)	|	Acc: 54.6%(train)
	Loss: 0.0078(valid)	|	Acc: 53.3%(valid)
